## 读取训练数据

### XLSX 转 CSV

In [3]:
import pandas as pd
# data = pd.read_excel('train.xlsx',index_col=0) # index_col=0如果不设置，转换后csv文件第一列就会是索引0，1，2...
# data.to_csv('real_train.csv',encoding='utf-8')

In [77]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import roc_auc_score


def do_metrics(y_test,y_pred):
    # print ("metrics.accuracy_score:")
    # print (metrics.accuracy_score(y_test, y_pred))
    # print ("metrics.confusion_matrix:")
    # print (metrics.confusion_matrix(y_test, y_pred))
    # print ("metrics.precision_score:")
    # print (metrics.precision_score(y_test, y_pred))
    # print ("metrics.recall_score:")
    # print (metrics.recall_score(y_test, y_pred))
    # print ("metrics.f1_score:")
    # print (metrics.f1_score(y_test,y_pred))
    # confusion_mat = confusion_matrix(y_test, y_pred)
    #print("metrics.auc_score:")
    #print(roc_auc_score(y_test, y_pred))
    return roc_auc_score(y_test, y_pred)

### 读Train数据

In [349]:
df_train = pd.read_csv('real_train.csv')

# count_2 = (df_train ==2.0).astype(int).sum(axis=1).to_list()
# df_train['count_2'] = count_2

df_train = df_train.replace({'LGP_HLD_CARD_LVL': {'A':1,'B':2,'C':3,'D':4,'E':5,'F':6},'NB_CTC_HLD_IDV_AIO_CARD_SITU': {'A':1,'B':2,'C':3,'D':4,'E':5,'F':6},'WTHR_OPN_ONL_ICO': {'A':1,'B':2},'MON_12_CUST_CNT_PTY_ID': {'Y':1}})


ID = df_train['CUST_UID']
LABEL = df_train['LABEL']
#经过xgboost特征重要性输出，去除部分特征，留存36维
train_drop = [
 # 'AGN_CNT_RCT_12_MON',
 # 'MON_12_EXT_SAM_NM_TRSF_OUT_CNT',
 # 'MON_12_EXT_SAM_AMT',
 # 'MON_12_TRX_AMT_MAX_AMT_PCTT',
 # 'MON_12_AGV_ENTR_ACT_CNT',
 # 'MON_12_AGV_LVE_ACT_CNT',
 # 'CUR_YEAR_PUB_TO_PRV_TRX_PTY_CNT',
 # 'MON_12_ACT_OUT_50_UP_CNT_PTY_QTY',
 # 'MON_12_ACT_IN_50_UP_CNT_PTY_QTY',
 # 'AI_STAR_SCO',
 # 'WTHR_OPN_ONL_ICO',
 # 'LGP_HLD_CARD_LVL',
 # 'NB_CTC_HLD_IDV_AIO_CARD_SITU',
 #'LABEL',
 'CUST_UID'
]

df_train.drop(train_drop,axis=1,inplace=True)




df_train

,LABEL,AGN_CNT_RCT_12_MON,ICO_CUR_MON_ACM_TRX_TM,NB_RCT_3_MON_LGN_TMS_AGV,AGN_CUR_YEAR_AMT,AGN_CUR_YEAR_WAG_AMT,AGN_AGR_LATEST_AGN_AMT,ICO_CUR_MON_ACM_TRX_AMT,COUNTER_CUR_YEAR_CNT_AMT,PUB_TO_PRV_TRX_AMT_CUR_YEAR,...,WTHR_OPN_ONL_ICO,EMP_NBR,REG_CPT,SHH_BCK,HLD_DMS_CCY_ACT_NBR,REG_DT,LGP_HLD_CARD_LVL,OPN_TM,NB_CTC_HLD_IDV_AIO_CARD_SITU,HLD_FGN_CCY_ACT_NBR
0,0,?,72,425.3,?,?,?,25879985.3,2,660732,...,2,2,1200000002,32,12,1708.45,?,416.84,?,2
1,0,2282,222,?,11776572.4,?,1964626.4,14755499.8,2,126608.3,...,2,1002,5000002,32,22,199.42,6,195.87,4,2
2,0,?,2,?,?,?,4599822.3,2,2,2,...,2,2,?,2,12,1135.55,6,1122,NaN,2
3,1,?,?,22,?,?,?,?,2,2,...,1,2,500002,12,12,98.45,NaN,92,3,2
4,1,?,?,?,?,?,?,?,?,?,...,1,2,?,2,12,?,NaN,355.87,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,0,?,2,55.3,?,?,?,2,2,2,...,2,402,300000002,12,22,204.26,3,130.06,2,2
39996,0,?,72,145.3,?,?,?,16431457.9,2,2,...,2,2,500000002,12,12,176.52,?,160.71,?,2
39997,0,21532,1132,?,81027170.3,?,12772212.7,596254539.9,2,484869.1,...,2,5002,500000002,32,12,2265.55,4,2132.65,NaN,2
39998,1,?,12,128.7,?,?,?,488502,2,2,...,2,2,10000002,12,12,516.52,?,510.06,?,2


### 处理问号、缺失值等

In [353]:
df_train = df_train.replace('?',-0.1)#.replace(2.0,0)
count_question = (df_train == -0.1).astype(int).sum(axis=1).to_list()
df_train['count_q'] = count_question

count_question = (df_train == 2).astype(int).sum(axis=1).to_list()
df_train['count_2'] = count_question

df_train.drop(df_train.index[(df_train['count_q'] >= 45)], inplace=True)
df_train

,LABEL,AGN_CNT_RCT_12_MON,ICO_CUR_MON_ACM_TRX_TM,NB_RCT_3_MON_LGN_TMS_AGV,AGN_CUR_YEAR_AMT,AGN_CUR_YEAR_WAG_AMT,AGN_AGR_LATEST_AGN_AMT,ICO_CUR_MON_ACM_TRX_AMT,COUNTER_CUR_YEAR_CNT_AMT,PUB_TO_PRV_TRX_AMT_CUR_YEAR,...,REG_CPT,SHH_BCK,HLD_DMS_CCY_ACT_NBR,REG_DT,LGP_HLD_CARD_LVL,OPN_TM,NB_CTC_HLD_IDV_AIO_CARD_SITU,HLD_FGN_CCY_ACT_NBR,count_q,count_2
0,0,-0.1,72,425.3,-0.1,-0.1,-0.1,25879985.3,2,660732,...,1200000002,32,12,1708.45,-0.1,416.84,-0.1,2,6,1
1,0,2282,222,-0.1,11776572.4,-0.1,1964626.4,14755499.8,2,126608.3,...,5000002,32,22,199.42,6.0,195.87,4.0,2,2,3
2,0,-0.1,2,-0.1,-0.1,-0.1,4599822.3,2,2,2,...,-0.1,2,12,1135.55,6.0,1122,NaN,2,7,1
3,1,-0.1,-0.1,22,-0.1,-0.1,-0.1,-0.1,2,2,...,500002,12,12,98.45,NaN,92,3.0,2,7,0
4,1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,...,-0.1,2,12,-0.1,NaN,355.87,3.0,2,32,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,0,-0.1,2,55.3,-0.1,-0.1,-0.1,2,2,2,...,300000002,12,22,204.26,3.0,130.06,2.0,2,6,3
39996,0,-0.1,72,145.3,-0.1,-0.1,-0.1,16431457.9,2,2,...,500000002,12,12,176.52,-0.1,160.71,-0.1,2,7,1
39997,0,21532,1132,-0.1,81027170.3,-0.1,12772212.7,596254539.9,2,484869.1,...,500000002,32,12,2265.55,4.0,2132.65,NaN,2,2,3
39998,1,-0.1,12,128.7,-0.1,-0.1,-0.1,488502,2,2,...,10000002,12,12,516.52,-0.1,510.06,-0.1,2,11,1


### 增加新特征

In [376]:
# #去除？过多的数据
# df_train.drop(df_train.index[(df_train['count_q'] >= 30)], inplace=True)
# df_train

#去除高相关性特征
# Create correlation matrix
corr_matrix = X.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
print(to_drop)
# Drop features 
# df_train.drop(to_drop, axis=1, inplace=True)


['MON_12_AGV_TRX_CNT', 'MON_12_ACM_ENTR_ACT_CNT', 'MON_12_AGV_ENTR_ACT_CNT', 'MON_12_ACM_LVE_ACT_CNT', 'MON_12_AGV_LVE_ACT_CNT', 'MON_6_50_UP_LVE_ACT_CNT', 'LAST_12_MON_DIF_NM_MON_AVG_TRX_AMT_NAV', 'EMP_NBR', 'LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL2']


In [378]:
X_train = X_train.drop(to_drop,axis=1)
X_test = X_test.drop(to_drop,axis=1)

In [409]:
import pandas as pd

X = df_train.drop(['LABEL'],axis=1).astype('float')
Y = df_train['LABEL']



#重要特征
important = ['CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL',
 'CUR_MON_COR_DPS_MON_DAY_AVG_BAL',
 'LAST_12_MON_COR_DPS_DAY_AVG_BAL',
 'CUR_YEAR_COR_DPS_YEAR_DAY_AVG_INCR',
 'REG_DT',
 'LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL',
 'REG_CPT',
 'CUR_YEAR_MID_BUS_INC',
 'ICO_CUR_MON_ACM_TRX_AMT',
 'OPN_TM']

i =['CUR_MON_COR_DPS_MON_DAY_AVG_BAL',
 'CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL',
 'LAST_12_MON_COR_DPS_DAY_AVG_BAL15',
 'LAST_12_MON_COR_DPS_DAY_AVG_BAL14',
 'LAST_12_MON_COR_DPS_DAY_AVG_BAL',
 'CUR_YEAR_MID_BUS_INC',
 'LAST_12_MON_COR_DPS_DAY_AVG_BAL16',
 'LAST_12_MON_COR_DPS_DAY_AVG_BAL11',
 'LAST_12_MON_COR_DPS_DAY_AVG_BAL13',
 'OPN_TM',
 'REG_DT',
 'LAST_12_MON_COR_DPS_DAY_AVG_BAL12',]
# X.drop(drop_name,axis=1,inplace=True)

LGP_HLD_CARD_LVL = X['LGP_HLD_CARD_LVL'].fillna(0)
NB_CTC_HLD_IDV_AIO_CARD_SITU = X['NB_CTC_HLD_IDV_AIO_CARD_SITU'].fillna(0)
WTHR_OPN_ONL_ICO = X['WTHR_OPN_ONL_ICO'].fillna(0)
MON_12_CUST_CNT_PTY_ID = X['MON_12_CUST_CNT_PTY_ID'].fillna(0)

X['CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL1'] = X['CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL']*LGP_HLD_CARD_LVL

X['LAST_12_MON_COR_DPS_DAY_AVG_BAL11'] = X['LAST_12_MON_COR_DPS_DAY_AVG_BAL']*X['CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL']
X['LAST_12_MON_COR_DPS_DAY_AVG_BAL12'] = X['LAST_12_MON_COR_DPS_DAY_AVG_BAL']*X['CUR_YEAR_COR_DPS_YEAR_DAY_AVG_INCR']
X['LAST_12_MON_COR_DPS_DAY_AVG_BAL13'] = X['LAST_12_MON_COR_DPS_DAY_AVG_BAL']*X['ICO_CUR_MON_ACM_TRX_AMT']
X['LAST_12_MON_COR_DPS_DAY_AVG_BAL14'] = X['LAST_12_MON_COR_DPS_DAY_AVG_BAL']*X['REG_CPT']
X['LAST_12_MON_COR_DPS_DAY_AVG_BAL15'] = X['LAST_12_MON_COR_DPS_DAY_AVG_BAL']/X['LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL']
X['LAST_12_MON_COR_DPS_DAY_AVG_BAL16'] = X['LAST_12_MON_COR_DPS_DAY_AVG_BAL']/X['REG_DT']

X['LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL1'] = X['LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL']*LGP_HLD_CARD_LVL
X['LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL2'] = X['LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL']*WTHR_OPN_ONL_ICO

X['MON_12_ACM_ACT_CNT']=X['CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL']-X['CUR_MON_COR_DPS_MON_DAY_AVG_BAL']
# X['MON_12_AGV_ACT_CNT']=X['MON_12_AGV_LVE_ACT_CNT']-X['MON_12_AGV_ENTR_ACT_CNT']

# X['WTHR_OPN_ONL_ICO1']=X['WTHR_OPN_ONL_ICO'].apply(lambda x:-1 if x==2 else 0)
# X['AGN_CUR_YEAR_A1']=X['AGN_CUR_YEAR_WAG_AMT']-X['AGN_AGR_LATEST_AGN_AMT']

#X['CUR_YEAR_MID_BUS_INC1']=X['REG_DT']*X['OPN_TM'] 
#X['REG_CPT1'] = X['LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL']*MON_12_CUST_CNT_PTY_ID

#']*X['CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=7)


X['LABEL'] = Y
X = X.drop(to_drop,axis=1)
X = X.drop(drop_f,axis=1)
Y = X['LABEL']
X = X.drop(['LABEL'],axis=1)


In [410]:
ntrain = X.shape[0]
ntest  = df_test.shape[0]

# X_train.describe()
from scipy.special import erfinv
def hot_encoder(df, columns):
    one_hot = {c: list(df[c].unique()) for c in columns}
    for c in one_hot:
        for val in one_hot[c]:
            df[c+'_oh_' + str(val)] = (df[c].values == val).astype(np.int)
    return df

def scale_feat(df):
    scaler = StandardScaler()
    return pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

def rank_gauss(x):
    # x is numpy vector
    N = x.shape[0]
    temp = x.argsort()
    rank_x = temp.argsort() / N
    rank_x -= rank_x.mean()
    rank_x *= 2 # rank_x.max(), rank_x.min() should be in (-1, 1)
    efi_x = erfinv(rank_x) # np.sqrt(2)*erfinv(rank_x)
    efi_x -= efi_x.mean()
    return efi_x

def df_inputSwapNoise(df, p):
    n = df.shape[0]
    idx = list(range(n))
    swap_n = round(n*p)
    for col in df.columns:
        arr = df[col].values
        col_vals = np.random.permutation(arr)
        swap_idx = np.random.choice(idx, size= swap_n)
        arr[swap_idx] = np.random.choice(col_vals, size = swap_n)
        df[col] = arr
    return df

print('Transforming data')

# df_all = X_train
# df_all_org = df_all.copy()
# df_all_noise = df_inputSwapNoise(df_all, 0.15)
# data_all_org = df_all_org.values
# data_all_noise = df_all_noise.values
# cols = data_all_org.shape[1]


df_all = pd.concat([X, df_test])
df_all_org = df_all.copy()
df_all_noise = df_inputSwapNoise(df_all, 0.15)
data_all_org = df_all_org.values
data_all_noise = df_all_noise.values
cols = data_all_org.shape[1]
print('Final data with {} columns'.format(cols))

for i in range(cols):
    u = np.unique(data_all_org[:,i])
    if u.shape[0] > 3:
        data_all_org[:,i] = rank_gauss(data_all_org[:,i])

for i in range(cols):
    u = np.unique(data_all_noise[:,i])
    if u.shape[0] > 3:
        data_all_noise[:,i] = rank_gauss(data_all_noise[:,i])

train_data_orig = data_all_org[0:ntrain,:]
test_data_orig  = data_all_org[ntrain:,:]
train_data_noise = data_all_noise[0:ntrain,:]
test_data_noise  = data_all_noise[ntrain:,:]
print(train_data_orig.shape)
print(test_data_orig.shape)
print(train_data_noise.shape)
print(test_data_noise.shape)

print('Original data')
all_data = np.vstack((train_data_orig, test_data_orig))
print('Noise data')
all_data_noise = np.vstack((train_data_noise, test_data_noise))

from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.models import load_model
from tensorflow.keras.initializers import glorot_normal, Zeros, Ones
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import RMSprop
import tensorflow as tf

print('Creating neural net')
model = Sequential()
model.add(Dense(units=1500, input_dim = all_data.shape[1], kernel_initializer=glorot_normal()))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dense(units=1500, kernel_initializer=glorot_normal()))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dense(units=1500, kernel_initializer=glorot_normal()))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dense(all_data.shape[1])) 
model.add(Activation('linear'))

opt = keras.optimizers.Adam(lr=0.0009)
model.compile(loss='mse', optimizer=opt)

print('Training neural net')
###Train this for 120 epochs
epochs = 20
chck = ModelCheckpoint('keras_dae.h5', monitor='loss', save_best_only=True)
cb = [ EarlyStopping(monitor='loss', patience=100, verbose=2, min_delta=0), chck ]
model.fit(all_data_noise, all_data, batch_size=128, verbose=1, epochs=epochs, callbacks=cb)

print('Applying neural net')
train_data_transform = model.predict(train_data_orig)
test_data_transform = model.predict(test_data_orig)
print(train_data_transform.shape)
print(test_data_transform.shape)



Transforming data
Final data with 37 columns
(39889, 37)
(12000, 37)
(39889, 37)
(12000, 37)
Original data
Noise data
Creating neural net
Training neural net
Epoch 1/20
406/406 [==============================] - 18s 41ms/step - loss: 0.3853
Epoch 2/20
406/406 [==============================] - 17s 41ms/step - loss: 0.1146
Epoch 3/20
406/406 [==============================] - 16s 41ms/step - loss: 0.1074
Epoch 4/20
406/406 [==============================] - 17s 41ms/step - loss: 0.1018
Epoch 5/20
406/406 [==============================] - 16s 40ms/step - loss: 0.0973
Epoch 6/20
406/406 [==============================] - 16s 40ms/step - loss: 0.0950
Epoch 7/20
406/406 [==============================] - 16s 40ms/step - loss: 0.0896
Epoch 8/20
406/406 [==============================] - 16s 40ms/step - loss: 0.0860
Epoch 9/20
406/406 [==============================] - 16s 40ms/step - loss: 0.0829
Epoch 10/20
406/406 [==============================] - 16s 40ms/step - loss: 0.0805
Epoch 11/20

In [412]:
#特征重要性


from sklearn import preprocessing
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier
import numpy as np
from sklearn.model_selection import train_test_split,cross_val_score
import lightgbm as lgb


lgb_clf = lgb.LGBMClassifier(objective='binary',metric='auc',num_leaves=35,max_depth=6,learning_rate=0.05,seed=2018,
        colsample_bytree=0.8,subsample=0.9,n_estimators=1000)
lgb_model = lgb_clf.fit(train_data_transform, Y, eval_set=[(train_data_transform, Y)], early_stopping_rounds=200#,verbose=-1
                       )

y_pred = lgb_clf.predict_proba(test_data_transform)

y_pred = [i[1] for i in y_pred]

# print(do_metrics(y_test,y_pred))
# lgb_predictors = [i for i in X_train.columns]

# lgb_feat_imp = pd.Series(lgb_model.feature_importances_, lgb_predictors).sort_values(ascending=False)
# lgb_feat_imp
# # lgb_feat_imp.index
# drop_name = []

# for i in (lgb_feat_imp.index):
#     drop_name.append(i)
# drop_name
# to_drop

/home/luohy/source~/.bashrc/envs/common/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	training's auc: 0.928478
[2]	training's auc: 0.931808
[3]	training's auc: 0.933081
[4]	training's auc: 0.934138
[5]	training's auc: 0.93546
[6]	training's auc: 0.935685
[7]	training's auc: 0.936076
[8]	training's auc: 0.936521
[9]	training's auc: 0.937133
[10]	training's auc: 0.937444
[11]	training's auc: 0.937871
[12]	training's auc: 0.938159
[13]	training's auc: 0.938515
[14]	training's auc: 0.938878
[15]	training's auc: 0.939203
[16]	training's auc: 0.939375
[17]	training's auc: 0.939613
[18]	training's auc: 0.93978
[19]	training's auc: 0.940059
[20]	training's auc: 0.940261
[21]	training's auc: 0.940461
[22]	training's auc: 0.940626
[23]	training's auc: 0.940801
[24]	training's auc: 0.940918
[25]	training's auc: 0.941118
[26]	training's auc: 0.941235
[27]	training's auc: 0.941401
[28]	training's auc: 0.941613
[29]	training's auc: 0.941858
[30]	training's auc: 0.942062
[31]	training's auc: 0.942275
[32]	training's auc: 0.942402
[33]	training's auc: 0.942682
[34]	training's auc: 

In [345]:
df_b = pd.read_csv('testb.csv')
df_b = df_b.replace('?',-0.1)#.replace(2.0,0)
df_bcount_question = (df_b == -0.1).astype(int).sum(axis=1).to_list()
df_b['count_q'] = df_bcount_question
df_b['count_q'].describe()

count    12000.000000
mean         5.579583
std          5.615748
min          0.000000
25%          1.000000
50%          4.000000
75%          7.000000
max         49.000000
Name: count_q, dtype: float64

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold

# 创建一个用于得到不同训练集和测试集样本的索引的StratifiedKFold实例，折数为10
strtfdKFold = StratifiedKFold(n_splits=10)
#把特征和标签传递给StratifiedKFold实例
kfold = strtfdKFold.split(X_train, y_train)
#循环迭代，（K-1）份用于训练，1份用于验证，把每次模型的性能记录下来。
scores = []
for k, (train, test) in enumerate(kfold):
    XGBClassifier.fit(X_train.iloc[train, :], y_train.iloc[train])
    score = roc_auc_score(X_train.iloc[test, :], y_train.iloc[test])
    scores.append(score)
    print('Fold: %2d, Training/Test Split Distribution: %s, AUC: %.3f' % (k+1, np.bincount(y_train.iloc[train]), score))
print('\n\nCross-Validation auc: %.3f +/- %.3f' %(np.mean(scores), np.std(scores)))

In [382]:
# use feature importance for feature selection
import numpy as np
from numpy import loadtxt
from numpy import sort
import pandas as pd
from xgboost.sklearn import XGBRegressor
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import roc_auc_score
# load data
#dataset = loadtxt('pima-indians-diabetes.csv', delimiter=",")
#dataset=pd.read_csv('train.csv',encoding='gbk')
#feature1=X[np.r_[np.array(dataset.columns.values)]]######数据用feature，其中[]内为需要参与计算的特征数，例：想看第一到第十个因子对因变量的影响，则填[0:9]
# split data into X and y
# X = feature1#特征因子
# Y = dataset['LABEL']
# split data into train and test sets
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=7)
# fit model on all training data
model = XGBClassifier(learning_rate=0.1,subsample=0.8)
model.fit(X_train, y_train)
# make predictions for test data and evaluate
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
# Fit model using each importance as a threshold
thresholds = sort(model.feature_importances_)
for thresh in thresholds:
    # select features using threshold
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    select_X_train = selection.transform(X_train)
    # train model
    selection_model = XGBClassifier()
    selection_model.fit(select_X_train, y_train)
    # eval model
    select_X_test = selection.transform(X_test)
    y_pred = selection_model.predict(select_X_test)
    predictions = [round(value) for value in y_pred]
    predictions = y_pred
    accuracy = roc_auc_score(y_test, predictions)
    print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))
    print(selection.get_support())

Accuracy: 89.92%


/home/luohy/source~/.bashrc/envs/common/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/home/luohy/source~/.bashrc/envs/common/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/home/luohy/source~/.bashrc/envs/common/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Thresh=0.007, n=52, Accuracy: 86.06%
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]


/home/luohy/source~/.bashrc/envs/common/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/home/luohy/source~/.bashrc/envs/common/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Thresh=0.007, n=51, Accuracy: 85.81%
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True False  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]


...

/home/luohy/source~/.bashrc/envs/common/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/home/luohy/source~/.bashrc/envs/common/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Thresh=0.030, n=3, Accuracy: 85.51%
[False False False False False False False False False False False False
 False False False False False False False False False False False False
  True  True  True False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]


/home/luohy/source~/.bashrc/envs/common/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/home/luohy/source~/.bashrc/envs/common/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Thresh=0.084, n=2, Accuracy: 84.69%
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False  True  True False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]
Thresh=0.315, n=1, Accuracy: 81.79%
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False  True False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]


/home/luohy/source~/.bashrc/envs/common/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [390]:
[ ]

feat = '''False  True  True  True  True  True  True  True  True  True  True  False  False  True  True  True  False  True  True  True  False  True  False  False  True  True  True  False  False  True  True  True  True  True  True  True  True  True  False  False  True  True  True  True  True  False  True  False  True'''.split('  ')
feat = '''True  True  True  True  True  True  True  True  True  True  False  False  True  True  True  True  False  True  False  True  True  False  False  True  True  True  True  False  True  True  True  True  False  True  False  True  False  False  False  False  True  True  False  False  True  True  True  True  True  True  True  True'''.split('  ')


names = X_train.columns.to_list()
drop_f = []

for i in range(len(names)):
    #print(feat[i])
    if feat[i]=='False':
        drop_f.append(names[i])
drop_f

['MON_12_EXT_SAM_TRSF_OUT_AMT',
 'MON_12_EXT_SAM_NM_TRSF_OUT_CNT',
 'MON_12_TRX_AMT_MAX_AMT_PCTT',
 'CUR_YEAR_PUB_TO_PRV_TRX_PTY_CNT',
 'MON_12_ACT_OUT_50_UP_CNT_PTY_QTY',
 'MON_12_ACT_IN_50_UP_CNT_PTY_QTY',
 'CUR_YEAR_COR_DPS_YEAR_DAY_AVG_INCR',
 'WTHR_OPN_ONL_ICO',
 'SHH_BCK',
 'REG_DT',
 'LGP_HLD_CARD_LVL',
 'OPN_TM',
 'NB_CTC_HLD_IDV_AIO_CARD_SITU',
 'count_2',
 'CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL1']

In [392]:
X_train.drop(drop_f,axis=1,inplace=True)
X_test.drop(drop_f,axis=1,inplace=True)

In [394]:
X_train

,AGN_CNT_RCT_12_MON,ICO_CUR_MON_ACM_TRX_TM,NB_RCT_3_MON_LGN_TMS_AGV,AGN_CUR_YEAR_AMT,AGN_CUR_YEAR_WAG_AMT,AGN_AGR_LATEST_AGN_AMT,ICO_CUR_MON_ACM_TRX_AMT,COUNTER_CUR_YEAR_CNT_AMT,PUB_TO_PRV_TRX_AMT_CUR_YEAR,MON_12_EXT_SAM_TRSF_IN_AMT,...,HLD_FGN_CCY_ACT_NBR,count_q,LAST_12_MON_COR_DPS_DAY_AVG_BAL11,LAST_12_MON_COR_DPS_DAY_AVG_BAL12,LAST_12_MON_COR_DPS_DAY_AVG_BAL13,LAST_12_MON_COR_DPS_DAY_AVG_BAL14,LAST_12_MON_COR_DPS_DAY_AVG_BAL15,LAST_12_MON_COR_DPS_DAY_AVG_BAL16,LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL1,MON_12_ACM_ACT_CNT
10162,-0.1,282.0,405.3,-0.1,-0.1,-0.1,13606912.0,2.0,2.0,2.0,...,2.0,5.0,3.736787e+13,8.120653e+12,8.600414e+13,6.320623e+13,0.568421,23544.873906,3.335881e+07,3428591.9
2002,-0.1,-0.1,18.7,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,2.0,...,2.0,16.0,8.028938e+14,-3.206552e+16,-1.298835e+07,2.597671e+16,0.507018,113733.387741,0.000000e+00,-9186.2
33646,-0.1,22.0,75.3,-0.1,-0.1,-0.1,30017757.4,2.0,2.0,2002.0,...,2.0,6.0,4.856916e+13,1.499538e+13,1.895158e+14,6.313455e+14,0.063086,3049.889810,3.002291e+08,3607631.8
33698,6202.0,182.0,602.0,15000387.1,15000387.1,2809894.3,52718324.3,8863168.3,2.0,488778.8,...,2.0,2.0,5.190807e+14,-2.623939e+14,1.375172e+15,-2.608528e+06,0.416426,12800.645497,0.000000e+00,971869.8
21056,-0.1,22.0,58.7,-0.1,-0.1,-0.1,2433377.5,2.0,2.0,16000002.0,...,2.0,5.0,1.058731e+16,-1.357376e+14,2.986332e+14,1.104514e+17,0.197163,634920.399400,0.000000e+00,10876744.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34234,-0.1,492.0,485.3,-0.1,-0.1,-0.1,25567770.0,12000002.0,2.0,2.0,...,12.0,5.0,7.817826e+13,-8.295132e+13,2.678793e+14,2.095445e+14,0.417093,5577.590513,1.004785e+08,1615320.6
13963,-0.1,-0.1,-0.1,-0.1,-0.1,934266.3,-0.1,2.0,2.0,2.0,...,22.0,7.0,6.715708e+12,-5.038070e+13,-1.029258e+06,1.330049e+15,0.407982,11755.564617,1.009122e+08,-1202211.0
922,-0.1,32.0,145.3,-0.1,-0.1,-0.1,383002.0,2.0,2.0,2.0,...,2.0,5.0,1.658672e+13,1.340407e+13,1.231184e+12,3.214564e+13,0.350620,8641.299194,1.833647e+07,-32515.3
38574,1092.0,162.0,355.3,4986062.0,-0.1,4986062.0,31011702.0,2.0,2.0,172590002.0,...,2.0,5.0,4.983510e+13,2.123541e+14,1.049385e+15,6.767672e+15,0.275871,22562.818489,-1.226601e+07,-36975090.2


In [415]:
model1 = Sequential()
model1.add(Dense(32, input_dim=37, activation='relu')) 
model1.add(Dense(27, activation='relu'))
model1.add(Dense(20, activation='relu'))
model1.add(Dense(15, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.summary()
model1.fit(train_data_transform, Y,epochs=100)
from sklearn.metrics import classification_report
prediction = model1.predict(test_data_transform)
prediction
results = []
for i in range(len(prediction)):
	# if prediction[i][0] > 0.5:
	# 	p = 1
	# else: 
	# 	p = 0
	results.append(prediction[i][0])
len(results)
# do_metrics(y_test, results)


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_35 (Dense)             (None, 32)                1216      
_________________________________________________________________
dense_36 (Dense)             (None, 27)                891       
_________________________________________________________________
dense_37 (Dense)             (None, 20)                560       
_________________________________________________________________
dense_38 (Dense)             (None, 15)                315       
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 16        
Total params: 2,998
Trainable params: 2,998
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
1247/1247 [==============================] - 3s 2ms/step - loss: 0.3817 - accuracy: 0.8226
Epoch 2/100
124

12000

In [404]:
df_test = pd.read_csv('testb.csv')


ID = df_test['CUST_UID']

test_drop = list(set(drop_f)-set(['LABEL','CUST_UID']))



df_test = df_test.replace('?',-0.1).replace('',0)


df_test = df_test.replace({'LGP_HLD_CARD_LVL': {'A':1,'B':2,'C':3,'D':4,'E':5,'F':6},'NB_CTC_HLD_IDV_AIO_CARD_SITU': {'A':1,'B':2,'C':3,'D':4,'E':5,'F':6},'WTHR_OPN_ONL_ICO': {'A':1,'B':2},'MON_12_CUST_CNT_PTY_ID': {'Y':1}})
df_test = df_test.drop(['CUST_UID'],axis=1)
df_test = df_test.astype('float')


count_question = (df_test == -0.1).astype(int).sum(axis=1).to_list()
df_test['count_q'] = count_question

count_question = (df_test == 2).astype(int).sum(axis=1).to_list()
df_test['count_2'] = count_question



LGP_HLD_CARD_LVL = df_test['LGP_HLD_CARD_LVL'].fillna(0)
NB_CTC_HLD_IDV_AIO_CARD_SITU = df_test['NB_CTC_HLD_IDV_AIO_CARD_SITU'].fillna(0)
WTHR_OPN_ONL_ICO = df_test['WTHR_OPN_ONL_ICO'].fillna(0)
MON_12_CUST_CNT_PTY_ID = df_test['MON_12_CUST_CNT_PTY_ID'].fillna(0)

df_test['CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL1'] = df_test['CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL']*LGP_HLD_CARD_LVL

df_test['LAST_12_MON_COR_DPS_DAY_AVG_BAL11'] = df_test['LAST_12_MON_COR_DPS_DAY_AVG_BAL']*df_test['CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL']
df_test['LAST_12_MON_COR_DPS_DAY_AVG_BAL12'] = df_test['LAST_12_MON_COR_DPS_DAY_AVG_BAL']*df_test['CUR_YEAR_COR_DPS_YEAR_DAY_AVG_INCR']
df_test['LAST_12_MON_COR_DPS_DAY_AVG_BAL13'] = df_test['LAST_12_MON_COR_DPS_DAY_AVG_BAL']*df_test['ICO_CUR_MON_ACM_TRX_AMT']
df_test['LAST_12_MON_COR_DPS_DAY_AVG_BAL14'] = df_test['LAST_12_MON_COR_DPS_DAY_AVG_BAL']*df_test['REG_CPT']
df_test['LAST_12_MON_COR_DPS_DAY_AVG_BAL15'] = df_test['LAST_12_MON_COR_DPS_DAY_AVG_BAL']/df_test['LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL']
df_test['LAST_12_MON_COR_DPS_DAY_AVG_BAL16'] = df_test['LAST_12_MON_COR_DPS_DAY_AVG_BAL']/df_test['REG_DT']

df_test['LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL1'] = df_test['LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL']*LGP_HLD_CARD_LVL
df_test['LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL2'] = df_test['LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL']*WTHR_OPN_ONL_ICO

df_test['MON_12_ACM_ACT_CNT']=df_test['CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL']-df_test['CUR_MON_COR_DPS_MON_DAY_AVG_BAL']

df_test = df_test.drop(to_drop,axis=1)
df_test = df_test.drop(drop_f,axis=1)




0        0
1        0
2        0
3        1
4        1
        ..
39995    0
39996    0
39997    0
39998    1
39999    0
Name: LABEL, Length: 39889, dtype: int64

In [419]:
with open('result_dae.txt','w')as f:
    for i in range(len(results)):
        f.write(str(ID[i])+' '+str((y_pred[i]))+'\n')